In [ ]:
# general imports
import torch as th
import torch.nn.functional as F
from torch_geometric.loader import DataLoader as PygDataLoader
import torch
from torch.nn import CrossEntropyLoss
from torch_geometric.data import DataLoader
from dataloaderNCARS import NCarsBest
from model import GraphRes
from tqdm import tqdm

In [ ]:
%env CUDA_LAUNCH_BLOCKING=1

In [ ]:
# Set device to use for training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("using device:", device)
print(torch.cuda.get_device_name(0))
# Initialize dataset and data loader
dataset = NCarsBest('./data/storage/', mode='train')

batch_size = 64
loader = PygDataLoader(dataset, batch_size=batch_size, shuffle=True)
print("example datapoint:", dataset.get(0))
print(dataset.get(0).size())

classes = 2
# Initialize model
model_input_shape = th.tensor((120, 100) + (3, ), device=device)
print("INPUT SHAPE:", model_input_shape)
model = GraphRes('ncars', model_input_shape, classes).to(device)
# model = torch.load("ncars_trained.model")


criterion = CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
acc = []
all_losses = []
test_accuracy = []
test_loss = []

def train():
    model.train()
    losses = []
    correct = 0
    for data in tqdm(loader):
        data = data.to(device)
        # print(data)
        optimizer.zero_grad()
        output = model(data)
        # print(output.size())
        # break
        # y_hat = F.softmax(  output, dim=-1)
        y = F.one_hot(data.y, num_classes=classes).type(torch.cuda.FloatTensor)
        loss = criterion(output, y)
        losses.append(loss)
        loss.backward()
        optimizer.step()
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    
    acc.append(correct / (len(loader.dataset)))
    print(acc[len(acc) - 1])
    return losses

testdata = NCarsBest('./data/storage/', mode='test')
testloader = PygDataLoader(testdata, batch_size=batch_size, shuffle=True)
def test(model, loader):
    model.eval()

    correct = 0
    for data in tqdm(loader):
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
            y = F.one_hot(data.y, num_classes=classes).type(torch.cuda.FloatTensor)
            loss = criterion(out, y)
            pred = out.max(dim=1)[1]
            correct += pred.eq(data.y).sum().item()
    return loss, correct / len(loader.dataset)

# Run training loop for 10 epochs

for epoch in range(40):
    if epoch == 20:
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    print("computing epoch", epoch)
    trainloss = train()
    loss, acc2 = test(model, testloader)
    test_accuracy.append(acc2)
    test_loss.append(loss)
    all_losses.append(trainloss)
    print("test_accuracy: ", test_accuracy[epoch])
    print("test_loss: ", test_loss[epoch])

In [ ]:
torch.save(model, "ncaltech101.model")

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

fig, (ax1,ax2) = plt.subplots(2)

ax1.plot(range(len(acc)), acc)
ax1.set_title('Training Accuracy')

ax2.plot(range(len(test_accuracy)), test_accuracy)
ax2.set_title('Test Accuracy')
plt.tight_layout()
fig.show()